In [ ]:
import cv2

import pyrealsense2 as rs
import numpy as np

class DepthCamera:
    def __init__(self):
        # Configure depth and color streams
        self.pipeline = rs.pipeline()
        config = rs.config()

        # Get device product line for setting a supporting resolution
        pipeline_wrapper = rs.pipeline_wrapper(self.pipeline)
        pipeline_profile = config.resolve(pipeline_wrapper)
        device = pipeline_profile.get_device()
        device_product_line = str(device.get_info(rs.camera_info.product_line))

        config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
        config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)


        # Start streaming
        self.pipeline.start(config)

    def get_frame(self):
        frames = self.pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        if not depth_frame or not color_frame:
            return False, None, None
        return True, depth_image, color_image

    def release(self):
        self.pipeline.stop()

In [ ]:
import numpy as np
import cv2
import os

model_path = os.getenv('HOME') + '/aiffel/darknet-master/yolov4.weights'
list_path = os.getenv('HOME') + '/aiffel/darknet-master/cfg/yolov4.cfg'
name_path = os.getenv('HOME') + '/aiffel/darknet-master/data/coco.names'

YOLO_net = cv2.dnn.readNet(model_path, list_path)

classes = []
with open(name_path, "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = YOLO_net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in YOLO_net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [ ]:
np.set_printoptions(suppress=True)

# Initialize Camera Intel Realsense
dc = DepthCamera()

def show_loc(x, y, color = (0, 0, 255)):
    cv2.circle(color_frame, (x, y), 4, color)
    
    distance = depth_frame[y, x]
    cv2.putText(color_frame, "{}mm".format(distance), (x, y - 20), cv2.FONT_HERSHEY_PLAIN, 2, color, 2)
    
def find_min_depth(depth_frame):
    flatten_list = sorted(list(set(np.array(depth_frame).flatten().tolist())))
    
    try:
        min_value = flatten_list[1]       
        return min_value
    except:
        return 150

while True:
    # 웹캠 프레임
    ret, depth_frame, color_frame = dc.get_frame()
    h, w, c = color_frame.shape

    # YOLO 입력
    blob = cv2.dnn.blobFromImage(color_frame, 0.00392, (640, 480), (0, 0, 0), True, crop=False)
    YOLO_net.setInput(blob)
    outs = YOLO_net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    ROI = []

    for out in outs:

        for detection in out:

            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * w)
                center_y = int(detection[1] * h)
                dw = int(detection[2] * w)
                dh = int(detection[3] * h)
                # Rectangle coordinate
                x = int(center_x - dw / 2)
                y = int(center_y - dh / 2)
                boxes.append([x, y, dw, dh])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.45, 0.4)


    for i in range(len(boxes)):
        if i in indexes:
            b_x, b_y, b_w, b_h = boxes[i]

            label = str(classes[class_ids[i]])
            score = confidences[i]
            
            if label == 'person':
                
                width = b_x + b_w
                hight = b_y + b_h
                
                if width >= 640:
                    width = 640
                
                if hight >= 480:
                    hight = 480
                    
                if b_y < 0:
                    b_y = 0
                
                if b_x < 0:
                    b_x = 0
                
                roi = depth_frame[b_y:hight, b_x:width]
                croi = color_frame[b_y:hight, b_x:width]
                min_depth = find_min_depth(roi)
                
                ROI.append(croi)
                                
                c_x = b_x + b_w//2
                c_y = b_y + b_h//2
                
                # 경계상자와 클래스 정보 이미지에 입력
                cv2.rectangle(color_frame, (b_x, b_y), (b_x + b_w, b_y + b_h), (0, 0, 255), 2)
                cv2.putText(color_frame, label, (b_x, b_y - 20), cv2.FONT_ITALIC, 0.5, (255, 255, 255), 1)
                
                cv2.putText(color_frame, str(min_depth), (c_x, c_y - 40), cv2.FONT_ITALIC, 1, (255, 255, 255), 1)
                show_loc(c_x, c_y, (255, 255, 255))


                
    ROI_distance = np.array((depth_frame[320,160], depth_frame[320,320], depth_frame[320,480], 
                    depth_frame[400,160], depth_frame[400,320], depth_frame[400,480]),dtype=np.float64)
    
    cv2.circle(color_frame, (480, 400), 4, (0, 0, 255))
    cv2.circle(color_frame, (400, 479), 4, (0, 255, 0))
    #diff_ROI = exROI - ROI_distance
    
    #exROI = ROI_distance
        
    print(ROI_distance)
                
    #cv2.imshow("YOLOv4", color_frame)
    cv2.imshow("Y", depth_frame)
    #cv2.imshow("YOLOv4", color_frame)
    #cv2.imshow("YOLOv4_depth", depth_frame)
    #for i in range(len(ROI)):
    #    cv2.imshow("ROI{}".format(i), ROI[i])

    key = cv2.waitKey(1)
    if key == 27:
        dc.release()
        cv2.destroyAllWindows()
        break

